In [56]:
import csv
import pandas as pd
import requests
import time
import random
import math
import requests
import json

AUTH_KEY = "ecc34701-f8e9-456b-8ac9-1e36dda61701"
ORG_ID = "d2084ccd-986b-4c42-a871-f65245365bbf"

def getAllNodes():
    url = "https://status.botcentralapi.com/service-monitoring-0.1/sysadmin/nodejs/instance/status/"

    headers = {
        'authorization': AUTH_KEY,
        'organizationid': ORG_ID,
        'content-type' : 'application/json'
        }

    response = requests.request("GET", url, headers=headers)
    return response.json()

def stopNode(id, accountId, accountUser):
    url = "https://status.botcentralapi.com/service-monitoring-0.1/sysadmin/nodejs/instance/stop/"+id
    payload = json.dumps({ "lpAccountId":accountId,"lpAccountUser":accountUser})
    headers = {
        'authorization': AUTH_KEY,
        'organizationid': ORG_ID,
        'content-type': "application/json"
        }
    response = requests.request("PUT", url, data=payload, headers=headers)
    print('[stopNode]',response.text)
def startNode(id, accountId, accountUser):
    url = "https://status.botcentralapi.com/service-monitoring-0.1/sysadmin/nodejs/instance/start/"+id
    payload = json.dumps({ "lpAccountId":accountId,"lpAccountUser":accountUser})
    headers = {
        'authorization': AUTH_KEY,
        'organizationid': ORG_ID,
        'content-type': "application/json"
        }

    response = requests.request("PUT", url, data=payload, headers=headers)

    print('[startNode]',response.text)
import time
def restartBot(botId, accountId, accountUser):
    print('='*50)
    print(botId, accountId, accountUser)
    print('-'*50)
    stopNode(botId, accountId, accountUser)
    time.sleep(1) # sleep for 1 second to give time for stop
    print()
    startNode(botId, accountId, accountUser)
def restartBotsOnServer(serverName, nodes_by_server):
    if not serverName in nodes_by_server:
        return
    print('#'*30)
    print(serverName)
    for n in nodes_by_server[serverName]:
        print(n['botName'],n['botId'], n['lpAccountId'],n['lpAccountUser'])
        #restartBot(n['botId'], n['lpAccountId'],n['lpAccountUser'])    

In [43]:
def filterNodesForChatType(nodes):
    return [node for node in nodes if 'type' in node and node['type'] == 'chat']
def filterNodesForProduction(nodes):
    return [node for node in nodes if 'environment' in node and node['environment'] == 'prod']
def filterNodesForOnlineNodes(nodes):
    return [node for node in nodes if 'nodeStatus' in node and node['nodeStatus'] == 'online']
def bucketByServers(nodes):
    node_dict = {}
    for node in nodes:
        if 'serverName' in node:
            if not node['serverName'] in node_dict:
                node_dict[node['serverName']] = []
            node_dict[node['serverName']].append(node)
    print(node_dict.keys())
    return node_dict

In [18]:
nodes = getAllNodes()
all_node_status = nodes['statusList']
online_chat_prod_nodes = filterNodesForOnlineNodes(filterNodesForProduction(filterNodesForChatType(all_node_status)))

In [59]:
[(n['botName'],n['botId'], n['lpAccountId'],n['lpAccountUser']) for n in online_chat_prod_nodes]

[('CA Cert Bot French',
  'e5cc1907259275e49161e7834c46caad8addc511',
  '65948500',
  'RoutingBot46'),
 ('SAP CHK (PRD)',
  'cd881e5b25d3cd15646f255a62ed635bf4234122',
  '81933160',
  'Concierge_Bot_Checkout_Primary'),
 ('SAP CHK (PRD)',
  'cd881e5b25d3cd15646f255a62ed635bf4234122',
  '81933160',
  'Concierge_Bot_Checkout_Secondary'),
 ("PROD - Lowe's Bathroom Bot",
  'd4e6187ce16fa356709ba545fd67d33ff4f86806',
  '22554410',
  'bathroom_bot1'),
 ("PROD - Lowe's Bathroom Bot",
  'd4e6187ce16fa356709ba545fd67d33ff4f86806',
  '22554410',
  'bathroom_bot2'),
 ('Outplex Bot',
  'c50214353d16bce44261ca7f6177ecf8f77c3876',
  '44778652',
  'outplex_bot'),
 ('CA Buick French',
  'f3ea213ccfdc55f6a59354e13969cb436f2ee242',
  '65948500',
  'RoutingBot40'),
 ("PROD - Lowe's Window Bot",
  'fdc63788fbfbfc7ed8d5475df35a51a111325fb9',
  '22554410',
  'window_bot1'),
 ("PROD - Lowe's Window Bot",
  'fdc63788fbfbfc7ed8d5475df35a51a111325fb9',
  '22554410',
  'window_bot2'),
 ('Holden US & AU Bot',
  '0

In [ ]:
# sample from a demo bot
restartBot('9bcf19cabeb47cd992f367ab73756608532e0426', '21055098','testbotcentral')

In [57]:
# CAREFUL WHEN RUNNING THIS IF restartBot(n['botId'], n['lpAccountId'],n['lpAccountUser'])  is uncommented
# bucket nodes by server
nodes_by_server = bucketByServers(online_chat_prod_nodes)
    
# run it for all prod servers: Prod_Node_Server_2', 'Prod_Node_Server_3', 
# 'Prod_Node_Server_5', 'Prod_Node_Server_7', 'Prod_Node_Server_4', 'Prod_Node_Server_6
restartBotsOnServer('Prod_Node_Server_3', nodes_by_server)

dict_keys(['Prod_Node_Server_2', 'Prod_Node_Server_3', 'Prod_Node_Server_5', 'Prod_Node_Server_7', 'Prod_Node_Server_4', 'Prod_Node_Server_6'])
##############################
Prod_Node_Server_3
SAP CHK (PRD) cd881e5b25d3cd15646f255a62ed635bf4234122 81933160 Concierge_Bot_Checkout_Primary
SAP CHK (PRD) cd881e5b25d3cd15646f255a62ed635bf4234122 81933160 Concierge_Bot_Checkout_Secondary
PROD - Lowe's Bathroom Bot d4e6187ce16fa356709ba545fd67d33ff4f86806 22554410 bathroom_bot1
PROD - Lowe's Window Bot fdc63788fbfbfc7ed8d5475df35a51a111325fb9 22554410 window_bot1
Vonie (RT AH) fedbf4265c766d1bff9dcec4089ab00e23dc8a29 45687643 routing_afterhours_bot
Vonie (RT) 895fd834833c1cbf9ef792a10bb6151a230907d4 45687643 routing_bot
PROD - Lowe's HVAC Bot 0bcdb2c102b21d5126b35096df7a97a862be5e25 22554410 hvac_bot1
Marketing Support Shopping 75414981a4b2fa80997b66a31d095669faa97887 65948500 RoutingBot9
Vonie (RT PCS CANADA) 469b1f46d6b89e90b7ca2c2934f9fab51e9effdd 45687643 pcs_canada_bot
PROD - Lowe's Hom